# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver
    - Chrome: chromedriver download
    - Firefox: geckodriver

In [28]:
# SELENIUM -> SEMPRE UTILIZAR ASPAS SIMPLES COM O XPATH

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2º plano:
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True # also works
# nav = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

#obs: métodos possiveis: 
# .click()
# .send_keys()

In [29]:
# pesquisar no google cotacao dolar
navegador.get("https://www.google.com.br/")
#copy full xpath
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar cotacao dolar
cotacaoDolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
display(cotacaoDolar)

'5.4345'

In [30]:
# pesquisar no google cotacao euro
navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE)
navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys("cotacao euro")
navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar cotacao euro
cotacaoEuro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
display(cotacaoEuro)

'6.533165692000001'

In [31]:
# pesquisar no google cotacao ouro
site = "https://www.melhorcambio.com/ouro-hoje"
navegador.get(site)

# pegar cotacao ouro
#cotacaoOuro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
# obs: o comando acima estava dando muito erro devido as aspas do cod, entao substitui-se por:
cotacaoOuro = navegador.find_element_by_id('comercial').get_attribute('value')
#trocar virgula por ponto para padronizar

cotacaoOuro = cotacaoOuro.replace(",",".")
display(cotacaoOuro)
navegador.quit()

'308.62'

### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [32]:
import pandas as pd

tabela_produtos = pd.read_excel("Aula 4 - Produtos.xlsx")
display(tabela_produtos)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [33]:
# Passo 1: atualizar as cotacoes
tabela_produtos.loc[tabela_produtos["Moeda"] == "Dólar", "Cotação"] = float(cotacaoDolar)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Euro", "Cotação"] = float(cotacaoEuro)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Ouro", "Cotação"] = float(cotacaoOuro)

# Passo 2: atualizar o preco base em real > cotacao * preco base original
tabela_produtos["Preço Base Reais"] = tabela_produtos["Cotação"] * tabela_produtos["Preço Base Original"]

# Passo 3: atualizar o preco final > preco base em real * ajuste
tabela_produtos["Preço Final"] = tabela_produtos["Preço Base Reais"] * tabela_produtos["Ajuste"]
tabela_produtos["Preço Final"] = tabela_produtos["Preço Final"].map("{:.2f}".format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.434500,5434.445655,1.40,7608.22
1,Carro Renault,4500.00,Euro,6.533166,29399.245614,2.00,58798.49
2,Notebook Dell,899.99,Dólar,5.434500,4890.995655,1.70,8314.69
3,IPhone,799.00,Dólar,5.434500,4342.165500,1.70,7381.68
4,Carro Fiat,3000.00,Euro,6.533166,19599.497076,1.90,37239.04
5,Celular Xiaomi,480.48,Dólar,5.434500,2611.168560,2.00,5222.34
6,Joia 20g,20.00,Ouro,308.620000,6172.400000,1.15,7098.26


### Agora vamos exportar a nova base de preços atualizada

In [34]:
tabela_produtos.to_excel("Produtos Tabela Atualizada.xlsx", index=False)
#index false para remover coluna indice